# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Locally Weighted Regression

In [2]:
import time

In [1]:
!pip install --upgrade bokeh==2.4.3

In [3]:
import numpy as np
from ipywidgets import interact
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from bokeh.io import push_notebook

output_notebook()

# Algorithm

In [4]:
%%time
def local_regression(x0, X, Y, tau):
    # add bias term
    x0 = np.r_[1, x0]
    X = np.c_[np.ones(len(X)), X]

    # fit model: normal equations with kernel
    xw = X.T * radial_kernel(x0, X, tau)
    beta = np.linalg.pinv(xw @ X) @ xw @ Y

    # predict value
    return x0 @ beta

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs


In [5]:
%%time
def radial_kernel(x0, X, tau):
    return np.exp(np.sum((X - x0) ** 2, axis=1) / (-2 * tau * tau))

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


# Run

## Data

In [6]:
%%time
n = 1000

# generate dataset
X = np.linspace(-3, 3, num=n)
Y = np.log(np.abs(X ** 2 - 1) + .5)

# jitter X
X += np.random.normal(scale=.1, size=n)

CPU times: user 506 µs, sys: 0 ns, total: 506 µs
Wall time: 515 µs


## Fit & Plot Models

In [7]:
%%time
def plot_lwr(tau):
    # prediction
    domain = np.linspace(-3, 3, num=300)
    prediction = [local_regression(x0, X, Y, tau) for x0 in domain]

    plot = figure(plot_width=400, plot_height=400)
    plot.title.text = 'tau=%g' % tau
    plot.scatter(X, Y, alpha=.3)
    plot.line(domain, prediction, line_width=2, color='red')

    return plot

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 12.2 µs


In [8]:
%%time
show(gridplot([
    [plot_lwr(10.), plot_lwr(1.)],
    [plot_lwr(0.1), plot_lwr(0.01)]
]))

CPU times: user 673 ms, sys: 6.56 ms, total: 679 ms
Wall time: 743 ms


## Interactive Model

In [9]:
%%time
def interactive_update(tau):
    model.data_source.data['y'] = [local_regression(x0, X, Y, tau) for x0 in domain]
    push_notebook()

domain = np.linspace(-3, 3, num=100)
prediction = [local_regression(x0, X, Y, 1.) for x0 in domain]

plot = figure()
plot.scatter(X, Y, alpha=.3)
model = plot.line(domain, prediction, line_width=2, color='red')
show(plot, notebook_handle=True)

CPU times: user 116 ms, sys: 781 µs, total: 117 ms
Wall time: 131 ms


In [10]:
%%time
interact(interactive_update, tau=(0.01, 3., 0.01))

interactive(children=(FloatSlider(value=1.5, description='tau', max=3.0, min=0.01, step=0.01), Output()), _dom…

CPU times: user 73.2 ms, sys: 1.87 ms, total: 75 ms
Wall time: 92.5 ms


<function __main__.interactive_update(tau)>

# The End